<a href="https://colab.research.google.com/github/dk-wei/ml-algo-implementation/blob/main/Dimensionality_Reduction_for_Data_Visualization_PCA_vs_TSNE_vs_UMAP_vs_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this Notebook, we will compare 4 main-stream Dimensionality Reduction algorithm:

- PCA
- TSNE
- UMAP
- LDA

source: https://towardsdatascience.com/dimensionality-reduction-for-data-visualization-pca-vs-tsne-vs-umap-be4aa7b1cb29

In [44]:
import numpy as np
import pandas as pd
import time

# For plotting
import plotly.io as plt_io
import plotly.graph_objects as go
%matplotlib inline

#PCA
from sklearn.decomposition import PCA

#TSNE
from sklearn.manifold import TSNE

#UMAP
import umap

#LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [45]:
# Load Data
train = pd.read_csv('https://raw.githubusercontent.com/gurpreet0610/sign_language_CNN/master/sign-language-mnist/sign_mnist_test.csv')
train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,6,149,149,150,150,150,151,151,150,151,152,152,152,152,152,153,153,151,152,152,153,152,152,151,151,150,150,150,149,150,150,150,152,152,151,152,152,152,152,152,...,131,134,144,147,125,87,87,103,107,110,116,113,75,74,74,74,76,74,82,134,168,155,146,137,145,146,149,135,124,125,138,148,127,89,82,96,106,112,120,107
1,5,126,128,131,132,133,134,135,135,136,138,137,137,138,138,139,137,142,140,138,139,137,137,136,135,134,133,134,132,129,132,134,135,135,137,139,139,139,140,141,...,114,112,89,48,133,194,182,185,184,184,182,181,172,174,177,178,178,179,181,183,187,175,165,154,118,107,100,75,96,83,47,104,194,183,186,184,184,184,182,180
2,10,85,88,92,96,105,123,135,143,147,152,157,163,168,171,182,172,175,185,183,184,185,185,185,183,183,182,181,178,86,88,93,96,108,125,137,145,149,154,160,...,145,123,78,162,239,227,229,226,226,225,224,222,89,91,94,111,136,154,167,184,125,3,166,225,195,188,172,185,161,122,68,166,242,227,230,227,226,225,224,222
3,0,203,205,207,206,207,209,210,209,210,209,208,207,207,209,208,210,210,207,209,209,208,209,210,209,207,208,209,207,206,208,209,208,208,210,211,210,211,209,209,...,85,80,84,151,238,255,255,250,237,245,250,232,103,101,102,103,95,208,231,227,209,190,179,182,152,150,159,119,83,63,154,248,247,248,253,236,230,240,253,255
4,3,188,191,193,195,199,201,202,203,203,203,204,204,204,203,202,198,216,217,135,181,200,195,194,193,190,189,187,185,190,194,196,197,200,202,204,206,207,207,206,...,93,52,24,53,63,33,41,51,48,45,49,55,149,150,150,148,147,151,124,82,84,81,69,81,111,103,84,75,53,28,26,40,64,48,29,46,49,46,46,53


In [46]:
#picking only the first 10 labels
train = train[train['label'] < 10]
# Setting the label and the feature columns
y = train.loc[:,'label'].values
x = train.loc[:,'pixel1':].values

## Implementing Plotting Functions

In [53]:
def plot_2d(component1, component2):
    
    fig = go.Figure(data=go.Scatter(
        x = component1,
        y = component2,
        mode='markers',
        marker=dict(
            size=20,
            color=y, #set color equal to a variable
            colorscale='Rainbow', # one of plotly colorscales
            showscale=True,
            line_width=1
        )
    ))
    fig.update_layout(margin=dict( l=100,r=100,b=100,t=100),width=2000/2,height=1200/2)                 
    fig.layout.template = 'plotly_dark'
    
    fig.show()

In [54]:
def plot_3d(component1,component2,component3):

  fig = go.Figure(data=[go.Scatter3d(
          x=component1,
          y=component2,
          z=component3,
          mode='markers',
          marker=dict(
              size=10,
              color=y,                # set color to an array/list of desired values
              colorscale='Rainbow',   # choose a colorscale
              opacity=1,
              line_width=1
          )
      )])
  
  # tight layout
  fig.update_layout(margin=dict(l=50,r=50,b=50,t=50),width=1800/2,height=1000/2)
  fig.layout.template = 'plotly_dark'
    
  fig.show()

## Standardizing the data

In [55]:
from sklearn.preprocessing import StandardScaler
## Standardizing the data
x = StandardScaler().fit_transform(x)

# PCA

In [56]:
start = time.time()
pca = PCA(n_components=3)
principalComponents = pca.fit_transform(x)
print('Duration: {} seconds'.format(time.time() - start))
principal = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2','principal component 3'])

Duration: 0.3482053279876709 seconds


## PCA 2D

In [57]:
plot_2d(principalComponents[:, 0],principalComponents[:, 1])

## PCA 3D

In [58]:
plot_3d(principalComponents[:, 0],principalComponents[:, 1],principalComponents[:, 2])

#  t-SNE

In [59]:
start = time.time()
pca_50 = PCA(n_components=50)
pca_result_50 = pca_50.fit_transform(x)
tsne = TSNE(random_state = 42, 
            n_components=3,
            verbose=0, 
            perplexity=40, 
            n_iter=400).fit_transform(pca_result_50)
print('Duration: {} seconds'.format(time.time() - start))

Duration: 37.13327217102051 seconds


## t-SNE 2D

In [60]:
plot_2d(tsne[:, 0],tsne[:, 1])

## t-SNE 3D

In [61]:
plot_3d(tsne[:, 0],tsne[:, 1],tsne[:, 2])

# UMAP

UMAP has different hyperparameters that can have an impact on the resulting embeddings:

- `n_neighbors`

This parameter controls how UMAP balances local versus global structure in the data. **This low values of n_neighbours forces UMAP to focus on very local structures while the higher values will make UMAP focus on the larger neighbourhoods**.

- `min_dist`

This parameter controls how tightly UMAP is allowed to pack points together. Lower values mean the points will be clustered closely and vice versa.

- `n_components`

This parameter allows the user to determine the dimensionality of the reduced dimension space.
- `metric`

This parameter controls how distance is computed in the ambient space of the input data.

In [62]:
start = time.time()
reducer = umap.UMAP(random_state=42,n_components=3)
embedding = reducer.fit_transform(x)
print('Duration: {} seconds'.format(time.time() - start))

Duration: 16.442603826522827 seconds


## UMAP 2D

In [63]:
plot_2d(reducer.embedding_[:, 0],reducer.embedding_[:, 1])

## UMAP 3D

In [64]:
plot_3d(reducer.embedding_[:, 0],reducer.embedding_[:, 1],reducer.embedding_[:, 2])

# LDA

In [65]:
start = time.time()
X_LDA = LDA(n_components=3).fit_transform(x,y)
print('Duration: {} seconds'.format(time.time() - start))

Duration: 1.1475765705108643 seconds


## LDA 2D

In [66]:
plot_2d(X_LDA[:, 0],X_LDA[:, 1])

## LDA 3D

In [67]:
plot_3d(X_LDA[:, 0],X_LDA[:, 1],X_LDA[:, 2])

# Summary

- **PCA** was not able to do such a good job in differentiating the signs. The main drawback of PCA is that it is highly influenced by outliers present in the data. PCA is a linear projection, which means it can’t capture non-linear dependencies, its goal is to find the directions (the so-called principal components) that maximize the variance in a dataset.

- **t-SNE** does a better job(it tries to preserve topology neighbourhood structure) as compared to PCA when it comes to visualising the different patterns of the clusters. Similar labels are clustered together, even though there are big agglomerates of data points on top of each other, certainly not good enough to expect a clustering algorithm to perform well.

- **UMAP** outperformed t-SNE and PCA, if we look at the 2d and 3d plot, we can see mini-clusters that are being separated well. It is very effective for visualizing clusters or groups of data points and their relative proximities. However, for this use case certainly not good enough to expect a clustering algorithm to distinguish the patterns.UMAP is much faster than t-SNE, another problem faced by the latter is the need for another dimensionality reduction method prior, otherwise, it would take a longer time to compute.

- Finally, **LDA** outperformed all the previous techniques in all aspects. Excellent computation time (second fastest) as well as proving the well-separated clusters we were expecting.

> Note: it is not surprising that LDA performed better than the other techiniques, that was what we were expecting . PCA, TSNE and UMAP are performed without the knowledge of the true class label, unlike LDA which is a **supervised learning**.

注意：
1. 线性判别LDA是有监督的，它需要用类标签来将不同的样本映射到不同的直线上，使得同类样本之间的类内距最小，不同样本的类间距最大。
2. 而主题模型LDA是无监督的。

![](https://miro.medium.com/max/1400/1*vXQ5sgMF0XmiY4Jc6gJVwA.png)
